# 4교시 조인 연산

### 목차
* [1. 조인 유형](#1.-조인-유형)
* [2. Inner Join](#2.-Inner-Join)
* [3. Outer Join](#3.-Outer-Join)
* [4. Left Semi Join](#3.-Outer-Join)
* [5. Left Anti Join](#3.-Outer-Join)
* [6. Natural Join](#3.-Outer-Join)
* [7. Cross Join - Cartesian Join](#7.-Cross-Join---Cartesian-Join)
* [8. 조인시 유의할 점](#4.-조인시-유의할-점)
* [참고자료](#참고자료)

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
spark

22/04/12 18:52:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## 1. 조인 유형

+ 스파크의 조인 타입
    + inner join, outer join, left outer join, right outer join
    + left semi join: 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 있는 경우에는 키가 일치하는 왼쪽 데이터셋만 유지 
    + left anti join: 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 없는 경우에는 키가 일치하지 않는 왼쪽 데이터셋만 유지
    + natural join: 두 데이터셋에서 동일한 이름을 가진 컬럼을 암시적으로 결합하는 조인을 수행
    + cross join: 왼쪽 데이터셋의 모든 로우와 오른쪽 데이터 셋의 모든 로우를 조합

![join](https://i.pinimg.com/originals/12/c8/90/12c890126fb2d1e99099468f53d47260.jpg)

![semi anti](https://miro.medium.com/max/1400/0*jfglHEcbPtwv50J1.png)

![](https://oracletechnocampus.files.wordpress.com/2016/02/2cea0-jointable2-bmp.jpg)

### JOIN 학습을 위해 상품은 단 하나만 구매할 수 있다고 가정하여 아래와 같은 테이블이 존재합니다
#### 정보 1. 고객은 4명이지만, 1명은 탈퇴하여 존재하지 않습니다
| 고객 아이디 (u_id) | 고객 이름 (u_name) | 고객 성별 (u_gender) |
| - | - | - |
| 1 | 정휘센 | 남 |
| 2 | 김싸이언 | 남 |
| 3 | 박트롬 | 여 |

#### 정보 2. 구매 상품은 3개이며, 탈퇴한 고객의 상품정보가 남아있습니다
| 구매 고객 아이디 (u_id) | 구매 상품 이름 (p_name) | 구매 상품 가격 (p_amount) |
| - | - | - |
| 2 | LG DIOS | 2,000,000 |
| 3 | LG Cyon | 1,800,000 |
| 4 | LG Computer | 4,500,000 |


In [3]:
user = spark.createDataFrame([
    (1, "정휘센", "남"),
    (2, "김싸이언", "남"),
    (3, "박트롬", "여")
]).toDF("u_id", "u_name", "u_gender")
user.printSchema()
display(user)
    
purchase = spark.createDataFrame([
    (2, "LG DIOS", 2000000),
    (3, "LG Cyon", 1800000),
    (4, "LG Computer", 4500000)
]).toDF("p_uid", "p_name", "p_amont")
purchase.printSchema()
display(purchase)

root
 |-- u_id: long (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)



u_id,u_name,u_gender
1,정휘센,남
2,김싸이언,남
3,박트롬,여


root
 |-- p_uid: long (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amont: long (nullable = true)



p_uid,p_name,p_amont
2,LG DIOS,2000000
3,LG Cyon,1800000
4,LG Computer,4500000


## 2. Inner Join

> join type 을 명시하지 않았을 때 기본적으로 내부조인을 수행함

* 왼쪽 테이블과 오른쪽 테이블에서 동일한 칼럼을 가져야함. 
* 참으로 평가되는 로우만 결합
* 세 번째 파라미터로 조인 타입을 명확하게 지정할 수 있음
* 두 테이블의 키가 중복되거나 여러 복사본으로 있다면 성능이 저하됨. 조인이 여러 키를 최소화하기 위해 일종의 카테시안 조인으로 변환되어 버림

### 2.1 구매 정보와 일치하는 고객 정보를 조인 (inner)

In [4]:
display(user.join(purchase, user.u_id == purchase.p_uid))
user.join(purchase, user.u_id == purchase.p_uid, "inner").count()

u_id,u_name,u_gender,p_uid,p_name,p_amont
3,박트롬,여,3,LG Cyon,1800000
2,김싸이언,남,2,LG DIOS,2000000


2

### <font color=green>1. [기본]</font> 고객 정보 f"{work_data}/tbl_user", 제품 정보 f"{work_data}/tbl_purchase" CSV 파일을 읽고
#### 1. 각각 스키마를 출력하세요
#### 2. 각각 데이터를 출력하세요
#### 3. 고객(tbl_user) 테이블의 u_id 와 제품(tbl_purchase) 테이블의 p_uid 는 고객아이디 입니다
#### 4. 고객 테이블을 기준으로 어떤 제품을 구매하였는지 inner join 을 통해 조인해 주세요
#### 5. 조인된 최종 테이블의 스키마와 데이터를 출력해 주세요

<details><summary>[실습1] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_user.csv")
)
left.printSchema()
left.show()

right = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_purchase.csv")
)
right.printSchema()
right.show()

join_codition = left.u_id == right.p_uid
answer = left.join(right, join_codition, "inner")
answer.printSchema()
display(answer)
```

</details>


In [1]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)

## 3. Outer Join

> 왼쪽과 오른쪽의 모든 로우를 제공. 

* 왼쪽이나 오른쪽 DataFrame에 일치하는 로우가 없다면 해당 위치에 null을 삽입
* 공통 로우가 거의 없는 테이블에서 사용하면 결과값이 매우 커지고 성능 저하

### 3.1 모든 고객의 정보 구매 정보를 조인 (left_outer)

In [6]:
user.join(purchase, user.u_id == purchase.p_uid, "left_outer").orderBy(purchase.p_uid.asc())

u_id,u_name,u_gender,p_uid,p_name,p_amont
1,정휘센,남,null,null,null
2,김싸이언,남,2,LG DIOS,2000000
3,박트롬,여,3,LG Cyon,1800000


### 3-2. 모든 상품에 대한 고객 정보를 조인 (right_outer)

In [7]:
user.join(purchase, user.u_id == purchase.p_uid, "right_outer").orderBy(purchase.p_uid.asc())

u_id,u_name,u_gender,p_uid,p_name,p_amont
2,김싸이언,남,2,LG DIOS,2000000
3,박트롬,여,3,LG Cyon,1800000
null,null,null,4,LG Computer,4500000


### 3-3. 모든 고객과 상품에 대한 정보를 조인 (full_outer)

In [8]:
user.join(purchase, user.u_id == purchase.p_uid, "full_outer").orderBy(purchase.p_uid.asc())

u_id,u_name,u_gender,p_uid,p_name,p_amont
1,정휘센,남,null,null,null
2,김싸이언,남,2,LG DIOS,2000000
3,박트롬,여,3,LG Cyon,1800000
null,null,null,4,LG Computer,4500000


## 4. Left Semi Join

> 오른쪽에 존재하는 것을 기반으로 왼쪽 로우만 제공

* 값이 존재하는 지 확인 용도, 값이 있다면 왼쪽 DataFrame에 중복 키가 있더라도 해당 로우는 결과에 포함
* 기존 조인 기능과는 달리 DataFrame의 필터 기능과 유사
* 하나의 테이블만 확실히 고려되고, 다른 테이블은 조인 조건만 확인하기 때문에 성능이 매우 좋음


In [9]:
user.join(purchase, user.u_id == purchase.p_uid, "left_semi").orderBy(user.u_id.asc())

u_id,u_name,u_gender
2,김싸이언,남
3,박트롬,여


## 5. Left Anti Join
+ 왼쪽 세미 조인의 반대 개념, 즉 오른쪽 DataFrame의 어떤 값도 포함하지 않음
+ SQL의 NOT IN과 같은 스타일의 필터

In [10]:
user.join(purchase, user.u_id == purchase.p_uid, "left_anti").orderBy(user.u_id.asc())

u_id,u_name,u_gender
1,정휘센,남


## 6. Natural Join
> 조인하려는 컬럼을 암시적으로 추정

* 암시적인 처리는 언제나 위험하므로 비추천
* Python join 함수는 이 기능을 지원하지 않음


In [11]:
user.createOrReplaceTempView("user")
purchase.createOrReplaceTempView("purchase")
spark.sql("show tables")

database,tableName,isTemporary
,purchase,true
,user,true


In [12]:
user.printSchema()
purchase.printSchema()

root
 |-- u_id: long (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)

root
 |-- p_uid: long (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amont: long (nullable = true)



In [13]:
# 지정된 필드의 값이 일치하는 경우 해당 필드를 기준으로 조인
spark.sql("SELECT * FROM user NATURAL JOIN purchase")

u_id,u_name,u_gender,p_uid,p_name,p_amont
1,정휘센,남,2,LG DIOS,2000000
1,정휘센,남,3,LG Cyon,1800000
1,정휘센,남,4,LG Computer,4500000
2,김싸이언,남,2,LG DIOS,2000000
2,김싸이언,남,3,LG Cyon,1800000
2,김싸이언,남,4,LG Computer,4500000
3,박트롬,여,2,LG DIOS,2000000
3,박트롬,여,3,LG Cyon,1800000
3,박트롬,여,4,LG Computer,4500000


## 7. Cross Join - Cartesian Join

> 교차 조인은 조건절을 기술하지 않은 내부 조인을 의미
* 왼쪽의 모든 로우를 오른쪽의 모든 로우와 결합함(결과의 로우 수 = 왼쪽 로우 수 * 오른쪽 로우 수)
* 큰 데이터에서 사용할 경우 out-of-memory exception 발생. 가장 좋지 않은 성능을 가진 조인. 주의해서 사용하며 특정 사례에서만 사용해야함.

In [14]:
""" 크로스 조인이지만 조건을 설정해야 하며, 조건에 부합된 결과를 출력하여 inner조인과 동일."""
joinExpr = user.u_id == purchase.p_uid
joinType = "cross"
user.join(purchase, on=joinExpr, how=joinType)

u_id,u_name,u_gender,p_uid,p_name,p_amont
3,박트롬,여,3,LG Cyon,1800000
2,김싸이언,남,2,LG DIOS,2000000


In [15]:
user.crossJoin(purchase)

u_id,u_name,u_gender,p_uid,p_name,p_amont
1,정휘센,남,2,LG DIOS,2000000
1,정휘센,남,3,LG Cyon,1800000
1,정휘센,남,4,LG Computer,4500000
2,김싸이언,남,2,LG DIOS,2000000
2,김싸이언,남,3,LG Cyon,1800000
2,김싸이언,남,4,LG Computer,4500000
3,박트롬,여,2,LG DIOS,2000000
3,박트롬,여,3,LG Cyon,1800000
3,박트롬,여,4,LG Computer,4500000


### <font color=green>2. [기본]</font> 고객 정보 f"{work_data}/tbl_user", 제품 정보 f"{work_data}/tbl_purchase" CSV 파일을 읽고
#### 1. 각각 스키마를 출력하세요
#### 2. 각각 데이터를 출력하세요
#### 3. 고객(tbl_user) 테이블의 u_id 와 제품(tbl_purchase) 테이블의 p_uid 는 고객아이디 입니다
#### 4. 모든 상품을 기준으로 구매하 고객정보를 조인해 주세요 (left: purchase, right: user, join: left_outer)
#### 5. 조인된 최종 테이블의 스키마와 데이터를 출력해 주세요
#### 6. 출력시에 상품 가격의 내림차순으로 정렬해 주세요

<details><summary>[실습2] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_purchase.csv")
)
left.printSchema()
# left.show()

right = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_user.csv")
)
right.printSchema()
# right.show()

join_codition = left.p_uid == right.u_id
answer = left.join(right, join_codition, "left_outer")
answer.printSchema()
display(answer.orderBy(desc("p_amount")))
```

</details>


In [2]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)

### <font color=blue>3. [중급]</font> 고객 정보 f"{work_data}/tbl_user", 제품 정보 f"{work_data}/tbl_purchase" CSV 파일을 읽고
#### 1. 각각 스키마를 출력하세요
#### 2. 각각 데이터를 출력하세요
#### 3. 고객(tbl_user) 테이블의 u_id 와 제품(tbl_purchase) 테이블의 p_uid 는 고객아이디 입니다
#### 4. 모든 고객을 기준으로 구매한 상품 정보를 조인해 주세요 (left: user, right: purchase, join: left_outer)
#### 5. 조인된 최종 테이블의 스키마와 데이터를 출력해 주세요
#### 6. 출력시에 상품 가격(tbl_purchase.p_amount)의 내림차순으로 정렬해 주세요
#### 7. 상품가격이 없는 경우에는 등록일자(tbl_user.u_signup)가 최신으로 정렬해 주세요

<details><summary>[실습3] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_user.csv")
)
left.printSchema()
# left.show()

right = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_purchase.csv")
)
right.printSchema()
# right.show()

join_codition = left.u_id == right.p_uid
answer = left.join(right, join_codition, "left_outer")
answer.printSchema()
display(answer.orderBy(desc("p_amount"), desc("u_signup")))
```

</details>


In [3]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)

### <font color=red>4. [고급]</font> 고객 정보 f"{work_data}/tbl_user", 제품 정보 f"{work_data}/tbl_purchase" CSV 파일을 읽고
#### 1. 각각 스키마를 출력하세요
#### 2. 각각 데이터를 출력하세요
#### 3. 고객(tbl_user) 테이블의 u_id 와 제품(tbl_purchase) 테이블의 p_uid 는 고객아이디 입니다
#### 4. 모든 고객을 기준으로 모든 상품 정보를 조인해 주세요 (left: user, right: purchase, join: inner)
#### 5. 조인된 최종 테이블의 스키마와 데이터를 출력해 주세요
#### 6. 출력시에 상품 가격(tbl_purchase.p_amount)의 내림차순으로 정렬해 주세요
#### 7. 상품가격이 없는 경우에는 등록일자(tbl_user.u_signup)가 최신으로 정렬해 주세요

<details><summary>[실습4] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_user.csv")
)
left.printSchema()
# left.show()

right = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_purchase.csv")
)
right.printSchema()
# right.show()

join_codition = left.u_id == right.p_uid
answer = left.join(right, join_codition, "inner")
answer.printSchema()
display(answer.orderBy(desc("p_amount"), desc("u_signup")))
```

</details>


In [4]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)

### <font color=blue>5. [중급]</font> 실습 5. 고객 정보 f"{work_data}/tbl_user", 제품 정보 f"{work_data}/tbl_purchase" CSV 파일을 읽고
#### 1. 각각 스키마를 출력하세요
#### 2. 각각 데이터를 출력하세요
#### 3. 고객(tbl_user) 테이블의 u_id 와 제품(tbl_purchase) 테이블의 p_uid 는 고객아이디 입니다
#### 4. 모든 고객과 모든 상품 정보를 조인해 주세요 (left: user, right: purchase, join: full_outer)
#### 5. 조인된 최종 테이블의 스키마와 데이터를 출력해 주세요
#### 6. 출력시에 상품 가격(tbl_purchase.p_amount)의 내림차순으로 정렬해 주세요
#### 7. 상품가격이 없는 경우에는 등록일자(tbl_user.u_signup)가 최신으로 정렬해 주세요

<details><summary>[실습5] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_user.csv")
)
left.printSchema()
# left.show()

right = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_purchase.csv")
)
right.printSchema()
# right.show()

join_codition = left.u_id == right.p_uid
answer = left.join(right, join_codition, "full_outer")
answer.printSchema()
display(answer.orderBy(desc("p_amount"), desc("u_signup")))
```

</details>


In [5]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)

### <font color=green>6. [기본]</font> 아래의 조인 연산 결과에서 null 값에 대한 치환을 해주세요
#### 1. 고객아이디(u_id)와 고객이름(u_name), 성별(u_gender), 가입일자(u_signup) 기본값을 채워주세요
##### u_id = 0, u_name = '미확인', u_gender = '미확인', u_signup = '19700101'

<details><summary>[실습6] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_purchase.csv")
)
left.printSchema()
# left.show()

right = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_user.csv")
)
right.printSchema()
# right.show()

join_codition = left.p_uid == right.u_id
user_fill = { "u_id":0, "u_name":"미확인", "u_gender":"미확인", "u_signup":"19700101" }
answer = left.join(right, join_codition, "left_outer").na.fill(user_fill)
answer.printSchema()
display(answer.orderBy(asc("u_signup")))
```

</details>


In [6]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)

## 8. 조인시 유의할 점

In [23]:
u = spark.createDataFrame([
    (1, "정휘센", "남"),
    (2, "김싸이언", "남"),
    (3, "박트롬", "여")
]).toDF("id", "name", "gender")
u.printSchema()
display(u)
    
p = spark.createDataFrame([
    (2, "LG DIOS", 2000000),
    (3, "LG Cyon", 1800000),
    (4, "LG Computer", 4500000)
]).toDF("id", "name", "amount")
p.printSchema()
display(p)

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)



id,name,gender
1,정휘센,남
2,김싸이언,남
3,박트롬,여


root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- amount: long (nullable = true)



id,name,amount
2,LG DIOS,2000000
3,LG Cyon,1800000
4,LG Computer,4500000


### 8.1 중복 컬럼명 처리가 되지 않은 경우
> #### AnalysisException: "Reference 'id' is ambiguous, could be: id, id.;"

In [26]:
up = u.join(p, u.id == p.id)
up.show()
# up.select("id") # 주석해제해서 오류 발생시키기

+---+--------+------+---+-------+-------+
| id|    name|gender| id|   name| amount|
+---+--------+------+---+-------+-------+
|  3|  박트롬|    여|  3|LG Cyon|1800000|
|  2|김싸이언|    남|  2|LG DIOS|2000000|
+---+--------+------+---+-------+-------+



### 3.2 중복 컬럼명 해결방안 - 데이터 프레임의 컬럼 명을 다르게 만든다

In [27]:
u1 = u.withColumnRenamed("id", "u_uid")
p1 = p.withColumnRenamed("id", "p_uid")
u1.printSchema()
p1.printSchema()

up = u1.join(p1, u1.u_uid == p1.p_uid)
display(up)
up.select("u_uid")

root
 |-- u_uid: long (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)

root
 |-- p_uid: long (nullable = true)
 |-- name: string (nullable = true)
 |-- amount: long (nullable = true)



u_uid,name,gender,p_uid,name,amount
3,박트롬,여,3,LG Cyon,1800000
2,김싸이언,남,2,LG DIOS,2000000


u_uid
3
2


### 3.3 중복 컬럼명 해결방안 - 조인 직후 중복 컬럼을 제거합니다

In [28]:
up = u.join(p, u.id == p.id).drop(p.id)
display(up)
display(up.select("id"))

id,name,gender,name,amount
3,박트롬,여,LG Cyon,1800000
2,김싸이언,남,LG DIOS,2000000


id
3
2


### <font color=red>7. [고급]</font> 고객 정보 f"{work_data}/tbl_user_id", 제품 정보 f"{work_data}/tbl_purchase_id" CSV 파일을 읽고
#### 1. *고객(tbl_user) 테이블의 아이디도 id 이고 제품(tbl_purchase) 테이블의 아이디도 id 입니다*
#### 2. 가장 비싼 제품을 구매한 고객의 고객정보와 제품정보를 출력해 주세요
#### 3. 최종 출력되는 컬럼은 고객아이디(u_id), 고객이름(u_name), 상품이름(p_name), 상품가격(p_amount) 이렇게 4개 컬럼입니다
#### 4. 상품가격 (p_amount) 내림차순으로 정렬되어야 합니다

<details><summary>[실습7] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_purchase_id.csv")
)
# left.printSchema()
# left.show()

right = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_user_id.csv")
)
# right.printSchema()
# right.show()

u_left = left.withColumnRenamed("id", "u_id")
u_right = right.withColumnRenamed("id", "p_uid")
u_left.printSchema()
u_right.printSchema()

join_codition = u_left.u_id == u_right.p_uid
answer = u_left.join(u_right, join_codition, "inner").where("p_amount > 0").select("u_id", "u_name", "p_name", "p_amount")
answer.printSchema()
display(answer.orderBy(desc("p_amount")))
```

</details>


In [7]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)

## 9. 참고. 스파크의 조인 수행 방식
- Join 을 수행할 사이즈에 따라 조인 수행하는 방식이 다릅니다. 주요한 조인 수행방식은 아래와 같습니다. 데이터의 분포와 파티셔닝을 고려한 성능 튜닝과 함께 자세한 부분은 고급과정에서 다룹니다. 
- Broadcast Hash Join : **SmallSize** join **AnySize**
- Shuffle hash join : **MiddleSize** join **LargeSize**
- Sort merge join : **LargeSize** join **LargeSize**
- BroadcastNestedLoopJoin

---

## 참고자료

* [Spark Programming Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)
* [PySpark SQL Modules Documentation](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html)
* <a href="https://spark.apache.org/docs/3.0.1/api/sql/" target="_blank">PySpark 3.0.1 Builtin Functions</a>
* [PySpark Search](https://spark.apache.org/docs/latest/api/python/search.html)
* [Pyspark Functions](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?#module-pyspark.sql.functions)
- [Spark SQL, DataFrames and Datasets Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)
- [Spark summit 2017 - Hive Bucketing in Apache Spark with Tejas Patil](https://youtu.be/6BD-Vv-ViBw?t=30) / [slide](https://www.slideshare.net/databricks/hive-bucketing-in-apache-spark-with-tejas-patil) / [한글 요약본](https://www.notion.so/Hive-Bucketing-in-Apache-Spark-Tejas-Patil-9374879e0ca744cc8e7047e82cf5fdfa)
- [Spark summit 2017 - Optimizing Apache Spark SQL Joins: Spark Summit East talk by Vida Ha](https://www.youtube.com/watch?v=fp53QhSfQcI) / [slide](https://www.slideshare.net/databricks/optimizing-apache-spark-sql-joins)
- [Everyday I'm Shuffling - Tips for Writing Better Apache Spark Programs](https://www.youtube.com/watch?v=Wg2boMqLjCg)
- [Spark Memory Management by 0x0fff](https://0x0fff.com/spark-memory-management/)
- [Apache Spark에서 컬럼 기반 저장 포맷 Parquet(파케이) 제대로 활용하기](http://engineering.vcnc.co.kr/2018/05/parquet-and-spark/)
- [Understanding Database Sharding](https://www.digitalocean.com/community/tutorials/understanding-database-sharding)